In [1]:
!conda install -y --channel https://conda.anaconda.org/CogSci pygame

Fetching package metadata: ......
Solving package specifications: .........

Package plan for installation in environment C:\Anaconda2:

The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.0.8                |           py27_0         231 KB
    pygame-1.9.2a0             |           py27_0         4.2 MB
    ------------------------------------------------------------
                                           Total:         4.4 MB

The following NEW packages will be INSTALLED:

    pygame: 1.9.2a0-py27_0

The following packages will be UPDATED:

    conda:  4.0.5-py27_0 --> 4.0.8-py27_0  

Fetching packages ...
conda-4.0.8-py   0% |                              | ETA:  --:--:--   0.00  B/s
conda-4.0.8-py   6% |##                             | ETA:  0:00:03  67.98 kB/s
conda-4.0.8-py  13% |####                           | ETA:  0:00:03  63.75 kB/s
conda-4.0.8-py  20% |######        

Using Anaconda Cloud api site https://api.anaconda.org


In [1]:
import pygame
from pygame.locals import *
# import 隨機亂數
from random import random, randint

pygame.init()


pygame.init()
font = pygame.font.SysFont("monospace", 32, bold=True)
# 結束的字型
font_gg = pygame.font.SysFont("monospace", 72, bold=True)

score_label = font.render("score: 0", 1, (255,255,255))

# 顯示生命
life_label =  font.render("life: 10", 1, (255,255,255))
# Game Over 文字
game_over_label =  font_gg.render("Game Over", 1, (255,100, 100))


# 隨機產生座標
def random_pos():
    return (randint(0,639), randint(0,479))


# 產生五個隨機的食物
food_pos = [random_pos() for i in range(5)]

# 產生隨機的敵人位置和方向
def random_enemy():
    if randint(0,1):
        direction = (-2,0)
        position = (700, randint(20, 460))
    else:
        direction = (0, -2)
        position = (randint(20, 620), 520)
    return direction, position
        
# 產生五個隨機敵人
enemy_dir = [] 
enemy_pos = []
for i in range(5):
    direction, position = random_enemy()
    enemy_dir.append(direction)
    enemy_pos.append(position) 

# 定義分數
score = 0



# 生命數量
life = 10

DISPLAYSURF = pygame.display.set_mode((640, 480))

# 設定視窗標題
pygame.display.set_caption('作業!')
# 取得時鐘 
fpsClock = pygame.time.Clock()
# 讀取聲音
eat_sound_effect = pygame.mixer.Sound('pacman_eatfruit.wav')
damage_sound_effect = pygame.mixer.Sound('pacman_eatghost.wav')
# 畫圖函數
def draw_rect(left, top, BOXSIZE, BOXSIZE2):
    pygame.draw.rect(DISPLAYSURF, (255,255,0), (left, top, BOXSIZE, BOXSIZE2))
    pygame.draw.circle(DISPLAYSURF, (0,0,0), (left+33,top+35), 10)
    pygame.draw.circle(DISPLAYSURF, (0,0,0), (left+68,top+35), 10)
    pygame.draw.arc(DISPLAYSURF, (255,0,0), (left+27,top+40, 50, 30), -3.14, 0, 3)    
    # 請填入你的畫圖程式碼
left=100
top=100
BOXSIZE=100
BOXSIZE2=100
dleft, dtop ,dBOXSIZE,dBOXSIZE2= 2, 2, 0, 0


# 設定一個變數 running 用來判斷是遊戲是否在跑
running = True
while running: 
    for event in pygame.event.get():
        if event.type == QUIT:
            pygame.quit()
            # 將 running 設成 False
            running = False
        elif event.type == KEYDOWN:
            if event.key == K_RIGHT: 
                dleft += 1
            elif event.key == K_LEFT:
                dleft -= 1
            elif event.key == K_UP:
                dtop -= 1
            elif event.key == K_DOWN:
                dtop += 1
        elif event.type == MOUSEBUTTONDOWN:
            dleft, dtop = event.pos[0]-left, event.pos[1]-top
            r = (dleft**2+dtop**2)**0.5
            if r!=0:
                dleft, dtop = dleft/r, dtop/r    
    if not running:
        break   
    # 更新位置    
    left+=dleft
    top+=dtop    
   
    if left > 540 or left < 0:
        dleft = -dleft+0.5*(random()-0.5)
    if top > 380 or top < 0:
        dtop = -dtop+0.5*(random()-0.5)
        
    new_food_pos = []
    for pos in food_pos:
        # 如果笑臉和食物的距離夠近, 就吃掉食物
        if 0 <(pos[0]-left)< 100 and 0 <(pos[1]-top)< 100:
            # 增加分數
            score += 1
            eat_sound_effect.play()
            score_label = font.render("score: %d"%score, 1, (255,255,255))
            # 生出一個新的食物
            pos = random_pos()
        # 將 pos 加入新的食物位置狀態
        new_food_pos.append(pos)
    # 將 food_pos 設定成處理後的狀態
    food_pos = new_food_pos    
    
    # 用顏色 (0,0,255) 填滿畫面
    DISPLAYSURF.fill( (0,0,255))
    
    # 處理敵人
    for i in range(5):
        direction = enemy_dir[i]
        position = enemy_pos[i]
        # 更新位置
        position = (position[0]+direction[0], position[1]+direction[1])
        # 超過邊界時, 製造新敵人
        if position[0] < 0 or position[1]<0:
            direction, position = random_enemy()
        # 和笑臉很接近時, 生命減少, 製造新敵人
        elif 0 <(position[0]-left)< 100 and 0 <(position[1]-top)< 100:
            damage_sound_effect.play()
            life -= 1
            life_label = font.render("life: %d"%life, 1, (255,255,255))
            direction, position = random_enemy()
        enemy_dir[i] = direction
        enemy_pos[i] = position
    draw_rect(int(left), int(top), BOXSIZE, BOXSIZE2)
    
    # 畫出敵人
    for pos in enemy_pos:
        pygame.draw.circle(DISPLAYSURF, (255,0, 0), pos, 5)
    
    # 生命值 <=0 時結束
    if life <= 0:
        life = dleft = dtop = 0
        DISPLAYSURF.blit(game_over_label, (150, 200))
    
    
    for pos in food_pos:
        pygame.draw.circle(DISPLAYSURF, (0,255,0), pos, 5)
        
    DISPLAYSURF.blit(life_label, (450, 10))

    # 顯示分數
    DISPLAYSURF.blit(score_label, (10, 10))
    pygame.display.update()
    fpsClock.tick(60)